# Tarea 4: traer su propio contenedor a SageMaker Studio

En este cuaderno, creará su propia imagen de Docker y creará un contenedor de procesamiento. Utilizará una clase **ScriptProcessor** del SDK de Python de Amazon SageMaker para ejecutar un script de preprocesamiento de scikit-learn dentro del contenedor. Luego, validará los resultados del procesamiento de datos que se guardan en Amazon Simple Storage Service (Amazon S3).

## Tarea 4.1: configuración del entorno

Instale las bibliotecas y las dependencias requeridas.

Configure un bucket de Amazon S3 para almacenar las salidas del trabajo de procesamiento y, además, obtenga el rol de ejecución para ejecutar el trabajo de procesamiento de SageMaker.

In [ ]:
%%capture

# add csv visualization and image-build packages
%pip install --upgrade sagemaker-datawrangler
%pip install sagemaker-studio-image-build 
%pip install --upgrade pandas

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np
import sagemaker_datawrangler
from sagemaker.s3 import S3Downloader

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'databucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

prefix = 'scripts/data'
S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/abalone_data.csv", local_path= 'data/')

## Tarea 4.2: creación de un contenedor de procesamiento

Defina y cree un contenedor de scikit-learn con el Dockerfile.

### Tarea 4.2.1: creación de un Dockerfile

Cree un directorio de Docker y agregue el Dockerfile que crea el contenedor de procesamiento. Como está creando un contenedor de scikit-learn, instala pandas y scikit-learn.

In [ ]:
%mkdir docker

In [ ]:
%%writefile docker/Dockerfile
FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye

RUN pip3 install pandas scikit-learn
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

### Tarea 4.2.2: creación de la imagen del contenedor

Cree una imagen del contenedor personalizada con la interfaz de la línea de comandos (CLI) para la creación de imágenes de Amazon SageMaker Studio.

Cuando usa la CLI para la creación de imágenes de Amazon SageMaker Studio, puede crear imágenes de Docker compatibles con Amazon SageMaker directamente desde sus entornos de SageMaker Studio. El uso de la CLI para la creación de imágenes ayuda a ahorrar tiempo y a aumentar la seguridad, ya que abstrae la creación del entorno de creación y requiere menos permisos.

Navegue hasta el directorio que contiene su Dockerfile y ejecute el comando de creación sm-docker. Este comando registra de forma automática la salida de la creación y devuelve el **URI de la imagen** de tu imagen de Docker. Este paso tarda entre 2 y 5 minutos en completarse.

In [ ]:
%%sh

rm /usr/lib/x86_64-linux-gnu/libstdc++.so.6

cp /opt/conda/lib/libstdc++.so.6 /usr/lib/x86_64-linux-gnu/libstdc++.so.6

cd docker

sm-docker build .

Cuando se completa la celda, se devuelve un URI de imagen parecido a *012345678910.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio-d-vcbyjgmmjzzy:data-scientist-test-user*.

1. Copie el **URI de la imagen** y péguelo en el editor de texto que prefiera. 

Use este **URI de la imagen** para crear una clase **ScriptProcessor**.

## Tarea 4.3: ejecución del trabajo de procesamiento de SageMaker

AnyCompany Consulting trabaja en un proyecto con un grupo de defensa de la naturaleza que estudia la edad del abulón. El abulón es un tipo de molusco o caracol marino. Quieren predecir la edad de los especímenes vivos en lugar de tener que abrir sus caparazones para determinar su edad.

El conjunto de datos sobre el abulón representa una población de más de 4000 abulones. El conjunto de datos incluye columnas para el sexo, la longitud, el diámetro, la altura, el peso entero, el peso descascarillado, el peso de las vísceras, el peso de la cáscara y los anillos.

Ejecute un trabajo de procesamiento en el conjunto de datos sobre el abulón.

In [ ]:
#import-data
shape=pd.read_csv("data/abalone_data.csv", header=0)
shape.sample(5)

Luego, use la clase ScriptProcessor de SageMaker para definir y ejecutar un script de procesamiento como un trabajo de procesamiento. Consulte [ScriptProcessor de SageMaker](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) para obtener más información sobre esta clase.

Para crear la clase ScriptProcessor, configure los siguientes parámetros:
- **base_job_name**: prefijo para el nombre del trabajo de procesamiento
- **command**: comando para ejecutar, además de cualquier marcador de la línea de comandos
- **image_uri**: URI de la imagen de Docker para usarlo para los trabajos de procesamiento
- **role**: rol de ejecución de SageMaker
- **instance_count**: cantidad de instancias para ejecutar el trabajo de procesamiento
- **instance_type**: tipo de instancia de Amazon Elastic Compute Cloud (Amazon EC2) que se usa para el trabajo de procesamiento

1. En el siguiente código, reemplace **REPLACE_IMAGE_URI** con el URI de su editor de texto.

In [ ]:
#sagemaker-script-processor
from sagemaker.processing import ScriptProcessor

# create a ScriptProcessor
script_processor = ScriptProcessor(
    base_job_name="own-processing-container",
    command=["python3"],
    image_uri="REPLACE_IMAGE_URI",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
)

A continuación, use el método ScriptProcessor.run() para ejecutar el script **sklearn_preprocessing.py** como un trabajo de procesamiento. Consulte [ScriptProcessor.run()](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor.run) para obtener más información sobre esta clase.

Para ejecutar el trabajo de procesamiento, configure los siguientes parámetros:
- **code**: ruta del script de preprocesamiento 
- **inputs**: ruta de datos de entrada para el script de preprocesamiento (ubicación de entrada de Amazon S3)
- **outputs**: ruta de salida para el script de preprocesamiento (ubicación de salida de Amazon S3)
- **arguments**: argumentos de línea de comandos para el script de preprocesamiento (como una relación de división para entrenamiento y prueba)

El tiempo para completar este trabajo de procesamiento es de aproximadamente 4 a 5 minutos.

In [ ]:
#processing-job
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "scripts/data"
output_preprocessed_data_prefix = "scripts/data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
script_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "abalone_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                        source="/opt/ml/processing/train",
                        destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                        source="/opt/ml/processing/test",
                        destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)
script_processor_job_description = script_processor.jobs[-1].describe()
print(script_processor_job_description)

## Tarea 4.4: validación de los resultados del procesamiento de datos

Valide la salida del trabajo de procesamiento que ejecutó mediante la observación de las primeras cinco filas de los conjuntos de datos de salida de entrenamiento y prueba.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

Cuando las celdas se completan, se devuelven respuestas parecidas a la siguiente salida:

```plain
Las 5 primeras filas de s3://databucket-us-east-1-xxxxxxx/scripts/data/output/validation/
M,0.55,0.425,0.155,0.918,0.278,0.243,0.335
I,0.5,0.4,0.12,0.616,0.261,0.143,0.194
M,0.62,0.48,0.155,1.256,0.527,0.374,0.318
I,0.22,0.165,0.055,0.055,0.022,0.012,0.02
M,0.645,0.5,0.175,1.511,0.674,0.376,0.378
```

Los encabezados de columna del conjunto de datos sobre el abulón son sexo (bebé, macho, hembra), longitud, diámetro, altura, peso_entero, peso_descascarillado, peso_vísceras, peso_cáscara y anillos. La salida de las carpetas **train/** y **validation/** muestra los datos procesados que se almacenan en el bucket de S3.

Logró crear su propio contenedor de procesamiento y usó SageMaker Processing para ejecutar el trabajo de procesamiento en ese contenedor personalizado.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.